## What is Quickvision ?

- Quickvision is a Computer Vision Library built on Top of Torchvision, PyTorch and Lightning

A brief description on how it works.

- It provides Easy to use torch native API, for fit(), train_step(), val_step() of models.

- It is Easily customizable and configurable models with various backbones.

- A complete torch native interface. All models are nn.Module all the training APIs are optional and not binded to models.

- Tensor First library. No abstraction and classes over it ! 

- A lightning API which helps to accelerate training over multiple GPUs, TPUs.

- A datasets API to common data format very easily and quickly to torch formats.

- A minimal package, with very low dependencies.



### Let's explore these with a simple Image Classification task.

## Install

Install directly from GitHub. Very soon it will be available over PyPi.

In [4]:
!pip install -q git+https://github.com/Quick-AI/quickvision.git

## Some imports

In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as T
from tqdm import tqdm
import pytorch_lightning as pl
# And finally our Hero !
import quickvision

## Create a dataset and dataloader

- This process is completely as in PyTorch.

- For example let's take the CIFAR10 dataset available in torchvision.


### Training and testing transfomrs

Quickivision is not binded to any library for transforms.

Let's keep it simple and use torchvision transforms for now.

In [8]:
# Train and validation Transforms which you would like
train_transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,))])
valid_transforms = T.Compose([T.ToTensor(), T.Normalize((0.5,), (0.5,))])

In [9]:
train_set = torchvision.datasets.CIFAR10("./data", download=True, train=True, transform=train_transforms)
valid_set = torchvision.datasets.CIFAR10("./data", download=True, train=False, transform=valid_transforms)

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [11]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=32, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=32, shuffle=False)

## Creating model !!

- Quickvision Provides simple functions to create models with pretrained weights.

In [13]:
from quickvision.models.classification import cnn

In [ ]:
# To create model with imagenet pretrained weights 
model = cnn.create_vision_cnn("resnet50", num_classes=10, pretrained="imagenet")

# Alternatively if you don't need pretrained weights
model_bare = cnn.create_vision_cnn("resnet50", num_classes=10, pretrained=None)

# It also supports other weights, do check a list which are supported !

model_ssl = cnn.create_vision_cnn("resnet50", num_classes=10, pretrained="ssl")

### Loss Function and Optimizer

- Again this is just like in torch

In [16]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

## Training 

### The usual boring PyTorch procedure.

- This too works with Quickvision, we just have models as `nn.Module` .

It is same way as you would train a CNN.


In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
model = model.to(device)
for epoch in range(2):
    for batch_idx, (inputs, target) in enumerate(train_loader):
        optimizer.zero_grad()
        inputs = inputs.to(device)
        target = target.to(device)
        out = model(inputs)
        loss = criterion(out, target)
        loss.backward()
        optimizer.step()

print("Done !!")
# And the boring boiler plate continues


Done !!


### The quickvision way of doing it !

- We have already implemented these boring procedures for you to speed up training !

In [19]:
model = cnn.create_vision_cnn("resnet50", num_classes=10, pretrained="imagenet")

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
history = cnn.fit(model=model, epochs=2, train_loader=train_loader,
        val_loader=valid_loader, criterion=criterion, device=device, optimizer=optimizer)

- If you need a granular control over the training.

You can use `train_step` and `val_step` methods.

We calculate commonly used metrics such as `accuracy` here for you !



In [ ]:
for epoch in tqdm(range(2)):
    print()
    print(f"Training Epoch = {epoch}")
    train_metrics = cnn.train_step(model, train_loader, criterion, device, optimizer)
    print()

    print(f"Validating Epoch = {epoch}")
    valid_metrics = cnn.val_step(model, valid_loader, criterion, device)


### Train with Lightning !

- We have the same logics implemented for PyTorch Lightning as well.
- This directly allows you to use all Lighning features such as Multi-GPU training, TPU Training, logging etc.

Quickly Prototype with Torch, transfer it to Lightning !



In [ ]:
model_imagenet = cnn.lit_cnn("resnet18", num_classes=10, pretrained="imagenet")

gpus = 1 if torch.cuda.is_available() else 0

# Again use all possible Trainer Params from Lightning here !!
trainer = pl.Trainer(gpus=gpus, max_epochs=2)
trainer.fit(model_imagenet, train_loader, valid_loader)

## Some more Features of Quickvision

- Support for mixed precision training in PyTorch API over GPUs.

- Supports Detection models such as `Detr`, `FRCNN`, `RetinaNet`.




# Conclusion

1. Quickvision allows you to bring your own `Dataset`, `Model` or `Code` Recipe

2. You may use models, training functions or Dataset loading utilites from quickvision.

3. Seamless API to connect with Lightning as well.

4. Faster Experimentation with same control with PyTorch or Lightning.

Visit us [here](https://github.com/Quick-AI/quickvision) over GitHub !

We are happy for new contributions / improvements to our package.

Quickivison is a library built for faster but no compromise PyTorch Training !

